Input Emotion6 Dataset

In [1]:
!pip install --upgrade kaggle
!touch kaggle.json
!echo "{\"username\":\"your_user_name\",\"key\":\"your_key\"}" >> kaggle.json
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json
!kaggle competitions download -c b3-data-information-course-vsa2
!unzip b3-data-information-course-vsa2.zip
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=18y8fUrK0Py3bvR_FdQLU8dKrTaRpUk0n
!gdown https://drive.google.com/uc?id=1KnDssH6LhFjJjEVgKebEUtZ4RzqyvvYl
!unzip Emotion6_dataset.zip
!unzip csv_emotion6.zip

# 教師データを弾く
import csv
import os
import glob
import shutil

folder_list = ['anger','disgust','fear','joy','sadness','surprise']

validation_data_path = "/content/sample_submission.csv"
test_data_path = "/content/Emotion6_test/"
for folder in folder_list:
  os.makedirs(test_data_path+folder)

with open(validation_data_path) as f:
  reader = csv.reader(f)
  for row in reader:
    if row[0] != "image":
      print(row[0])
      os.remove("/content/Emotion6/"+row[0])

# with open("/content/csv_emotion6/ground_truth_test.csv") as f:
#   reader = csv.reader(f)
#   for row in reader:
#     if row[0] != "image":
#      shutil.move("/content/Emotion6/"+row[0],"/content/Emotion6_test/"+row[0])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/1.50k [00:00<?, ?B/s]
100% 1.50k/1.50k [00:00<00:00, 1.55MB/s]
Archive:  b3-data-information-course-vsa2.zip
  inflating: sample_submission.csv   
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/uc?id=18y8fUrK0Py3bvR_FdQLU8dKrTaRpUk0n
To: /content/Emotion6_dataset.zip
100% 191M/191M [00:02<00:00, 70.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KnDssH6LhFjJjEVgKebEUtZ4RzqyvvYl
To: /content/csv_emotion6.zip
100% 12.7k/12.7k [00:00<00:00, 21.5MB/s]
Archive:  Emotion6_dataset.zip
  inflating: Emotion6/joy/294.jpg    
  inflating: Emotion6/joy/327.jpg    
  inflating: Emotion6/joy/309.jpg    
  inflating: Emotion6/joy/325.jpg   

In [ ]:
import tensorflow as tf
import numpy as np
import glob
import matplotlib.pyplot as plt

abs_path = '/content/Emotion6/'
folder_list = ['anger','disgust','fear','joy','sadness','surprise']
x = []
y = []
for i in range(0,6):
  picture_list = glob.glob(abs_path + folder_list[i] + "/*")
  for p in picture_list:
    buf = tf.keras.utils.img_to_array(tf.keras.utils.load_img(p, target_size=(100, 100)))
    x.append(buf)
    y.append([i])

train_rate = 0.8

p = list(zip(x,y))
np.random.shuffle(p)
x, y = zip(*p)

x_train = x[:int(len(x)*train_rate)]
y_train = y[:int(len(y)*train_rate)]
x_test = x[int(len(x)*train_rate):]
y_test = y[int(len(y)*train_rate):]

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

# x = np.array(x)
# y = np.array(y)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# base modelを用意
input = tf.keras.Input(shape=(None, None, 3))
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (160, 160)))(input)
x = tf.keras.layers.Lambda(tf.keras.applications.efficientnet_v2.preprocess_input)(x)

base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B2(
    weights='imagenet',
    input_tensor=x,
    input_shape=(160,160,3),
    include_top=False,
    pooling='avg'
)

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(len(folder_list), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=1e-4,momentum=0.9),
    metrics=['accuracy']
)

history = model.fit(
    x_train,
    y_train,
    batch_size=50,
    epochs=70,
    validation_data=(x_test,y_test)
)

scores = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
plt.plot(history.history['accuracy'], label="acc", ls="-", marker="o")
plt.plot(history.history["val_accuracy"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

pred = np.argmax(model.predict(x_test[0:3]), axis=1)

print(pred)
model.summary()
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
model.save("results"+'/model_1223_1.h5')

(1269, 100, 100, 3)
(1269, 1)
(318, 100, 100, 3)
(318, 1)
Epoch 1/70
26/26 [==============================] - 29s 335ms/step - loss: 1.8028 - accuracy: 0.1608 - val_loss: 1.7977 - val_accuracy: 0.1572
Epoch 2/70
26/26 [==============================] - 5s 198ms/step - loss: 1.8039 - accuracy: 0.1663 - val_loss: 1.7945 - val_accuracy: 0.1541
Epoch 3/70
26/26 [==============================] - 5s 198ms/step - loss: 1.8013 - accuracy: 0.1560 - val_loss: 1.7937 - val_accuracy: 0.1667
Epoch 4/70
26/26 [==============================] - 5s 199ms/step - loss: 1.8002 - accuracy: 0.1568 - val_loss: 1.7928 - val_accuracy: 0.1572
Epoch 5/70
26/26 [==============================] - 5s 200ms/step - loss: 1.7967 - accuracy: 0.1686 - val_loss: 1.7921 - val_accuracy: 0.1572
Epoch 6/70
26/26 [==============================] - 5s 203ms/step - loss: 1.7980 - accuracy: 0.1694 - val_loss: 1.7928 - val_accuracy: 0.1541
Epoch 7/70
26/26 [==============================] - ETA: 0s - loss: 1.7938 - accuracy: 0.